In [ ]:
from pyarrow import hdfs
fs = hdfs.connect()
import pandas as pd
import numpy as np
import seaborn as sns
from io import BytesIO, StringIO
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, classification_report, accuracy_score, confusion_matrix
import xgboost as xgb
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
from xgboost import plot_importance
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score, KFold
from sklearn.utils import shuffle
from itertools import chain
import pickle

pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.options.display.max_rows = 100

exec(fs.cat('/dsz/app/dsc/common/settings.py'))

sample = %sql select * from source.data_20181231

# Vyloučení dat z Cribis financial statements - jsou dostupná jen pro minimum klientů v rámci následného skórování
sample.drop(sample.columns[114:304],axis=1,inplace=True) 

# drop proměnných, které už nebyly v prediktorech k konci roku 2019
drop_chybejici_pred = ['anonymized_1', 'anonymized_2', 'anonymized_3',
                       'anonymized_4', 'anonymized_5', 'anonymized_6', 'anonymized_7', 'anonymized_8', 
                       'anonymized_9','anonymized_10','anonymized_11', 'anonymized_12','anonymized_13', 
                      'anonymized_14', 'anonymized_15', 'anonymized_16', 'anonymized_17', 
                       'anonymized_18', 'anonymized_19','anonymized_20', 'anonymized_21']

# Definice vlastních funkcí

# Test, zda model funguje i na firmách, které jsou klienti méně než rok
def firmy_do_roka(X_test,y_test,join,model,target):
    sjednoceni_1 = pd.merge(X_test,y_test, on=join)
    sjednoceni_1 = sjednoceni_1[sjednoceni_1['klient_do_roka']==1]
    sjednoceni_y_1 = sjednoceni_1[target]
    sjednoceni_X_1= sjednoceni_1.drop(columns=[target])
    print("Testovaci", accuracy_score(sjednoceni_y_1, model.predict(sjednoceni_X_1)))
    
# Zobrazení významnosti proměnných podle GAIN
def zobrazeni_promennych(X_train,model,hranice):
    list_features = list(zip(X_train.columns,model.feature_importances_.tolist()))
    df_features = pd.DataFrame(list_features).sort_values(by=[1], ascending=False)
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        features = df_features[df_features[1] > hranice]
        print(features)
        features = df_features[df_features[1] > hranice][0]   
    return features

def grid_search(model_type,X,y,est_min,est_max,step,depth_min,depht_max):
    vysledky_celkove = []
    n = 0
    for estimator in range(est_min,est_max,step):
        for depth in range(depth_min,depht_max):
            vysledky_celkove.append([estimator] + [depth] + [cross_val_score(model_type(n_estimators=estimator, max_depth=depth), X, y, cv = KFold(n_splits=5, shuffle=True, random_state=0), n_jobs= 10).mean()])
#             n += 1
#             print(n)
    vysledky = pd.DataFrame.from_records(vysledky_celkove)
    vysledky_test = vysledky.pivot(0,1,2)
    sns.heatmap(vysledky_test)

# Transformace

# stahneme hodnoty vsech kategorickych promennych

cat_vars_all_file = BytesIO(fs.cat('/user/FOP/notebooks/categories.csv'))
cat_vars_all_desc = pd.read_csv(cat_vars_all_file, sep = ';')
cat_vars_all_desc['variable_cat'] = cat_vars_all_desc['variable'] + '_cat'

from collections import defaultdict
# funkce, ktera vezme pandi dataset a dva sloupce (klice, hodnoty) a vrati slovnik klic: [hodnoty]
# v tomhle formatu ulozime nactene pripustne hodnoty kategorickych promennych
def pandas_to_dict_of_lists(df, key_col, val_col):
    lst = df[[key_col, val_col]].values.tolist()
    orDict = defaultdict(list)
    for key, val in lst:
        orDict[key].append(val)
    return dict(orDict)

# vyhození proměnných
moje_promenne = ['FOP_40']
ostatni_promenne = ['customer_key']
drop_sample = moje_promenne + ostatni_promenne
kategoricke_promenne_chybi = ['FOP_30','FOP_31']
kategoricke_promenne_drop = ['FOP_32','FOP_33','FOP_34','FOP_36']

# vyhození záporných targetů
sample = sample[sample['target'] >= 0]

sample = sample.drop(columns=drop_sample)#.set_index('customer_key_firma')

# slovnik {promenna_1: [pripustna_hodnota_1, ...], ...}
cat_vars_desc = pandas_to_dict_of_lists(df = cat_vars_all_desc, key_col = 'variable_cat', val_col = 'value_string')
cat_vars = cat_vars_desc.keys()
cat_vars = [e for e in cat_vars if e not in kategoricke_promenne_chybi] 
cat_vars_bez = [x[:-4] for x in cat_vars]
cat_vars = [e for e in cat_vars if e not in kategoricke_promenne_drop] 

vysledky_celkove = []
pocet = 0
n = 0

# logaritmovaný target
sample['target_log'] = np.log(sample['target'])
# rozbinovany target
bins1 = [-9.99E20, 10000000, 100000000, 220000000, 2000000000, 9.99E20]
labels1 = ['1','2','3','4','5']
sample['anonymized_63'] = pd.cut(sample['target'], bins=bins1, labels=labels1)
bins2 = [-9.99E20, 8000000, 15000000, 45000000, 80000000,200000000,250000000,2000000000, 9.99E20]
labels2 = ['1','2','3','4','5','6','7','8']
sample['anonymized_64'] = pd.cut(sample['target'], bins=bins2, labels=labels2)
# firma klientem necely rok
sample['klient_do_roka'] = (sample['doba_klient'] <= 365) 

# vyhození kategorických proměnných, které chybí v mém datasetu

cat_vars_moje = ['FOP_40','FOP_41','FOP_42','FOP_43','FOP_44','FOP_45','FOP_46','FOP_47','FOP_48','FOP_49']
cat_vars_moje = [e for e in cat_vars if e not in drop_sample] 
features_to_drop = cat_vars_bez + kategoricke_promenne_drop

cat_variables = cat_vars + cat_vars_moje

sample_dummies = sample.drop(columns=features_to_drop).set_index('customer_key_firma')
sample_dummies = pd.get_dummies(sample_dummies, columns=set(cat_variables)-set(features_to_drop),dummy_na=True, drop_first=True)

PO = sample_dummies['FOP'] == 'N'
FOP = sample_dummies['FOP'] == 'Y'
ROK_2018 = sample_dummies['platnost_obratu'] == '2018-12-31'
ROK_2017 = sample_dummies['platnost_obratu'] == '2017-12-31'

# Na radu Kuby Mariana jsem použil tuto funkci asi pro převod všeho na čísla
sample_dummies = sample_dummies.apply(pd.to_numeric, errors='coerce', axis=1)

# Modely - klientská i neklientská data

# ------------------------------------------PO------------------------------------------

# Rozbinovaný target

drop = ['FOP','target','target_log','anonymized_63','anonymized_64']

sample_dummies_PO = sample_dummies.loc[PO]

target_PO_bin_1 = sample_dummies.loc[PO]['anonymized_63']
target_PO_bin_2 = sample_dummies.loc[PO]['anonymized_64']
sample_dummies_PO_bin = sample_dummies_PO.drop(columns=drop)

X_train_PO_bin_1, X_test_PO_bin_1, y_train_PO_bin_1, y_test_PO_bin_1 = train_test_split(sample_dummies_PO_bin, target_PO_bin_1, random_state = 0)
X_train_PO_bin_2, X_test_PO_bin_2, y_train_PO_bin_2, y_test_PO_bin_2 = train_test_split(sample_dummies_PO_bin, target_PO_bin_2, random_state = 0)

# 5 Kategorií (0-10,10-100,100-220,220-2000,2000-)

# Model pro PO všechny proměnné s rozbinovaným targetem
xg_class_PO_1 = xgb.XGBClassifier(n_estimators=50, max_depth=5)
xg_class_PO_1.fit(X_train_PO_bin_1, y_train_PO_bin_1)
print("Trenovaci", accuracy_score(y_train_PO_bin_1, xg_class_PO_1.predict(X_train_PO_bin_1)))
print("Testovaci", accuracy_score(y_test_PO_bin_1, xg_class_PO_1.predict(X_test_PO_bin_1)))

firmy_do_roka(X_test_PO_bin_1,y_test_PO_bin_1,'customer_key_firma',xg_class_PO_1,'anonymized_63') 

# Zobrazení významnosti proměnných podle GAIN rozbinovaný target
features_PO_bin_1 = zobrazeni_promennych(X_train_PO_bin_1,xg_class_PO_1,0.005)
features_PO_bin_1 = [e for e in features_PO_bin_1 if e not in drop_chybejici_pred]
    
# Vytvoření trénovacího a testovacího vzorku jen z vybraných proměnných     
sample_dummies_PO_bin_select_1 = sample_dummies_PO_bin[sample_dummies_PO_bin.columns.intersection(features_PO_bin_1)]
X_train_PO_bin_s_1, X_test_PO_bin_s_1, y_train_PO_bin_s_1, y_test_PO_bin_s_1 = train_test_split(sample_dummies_PO_bin_select_1, target_PO_bin_1, random_state = 0)

# Model pro PO vybrané proměnné s rozbinovaným targetem, R^2 crossvalidační metodou 0,94
xg_class_PO_s_1 = xgb.XGBClassifier(n_estimators=50, max_depth=5)
xg_class_PO_s_1.fit(X_train_PO_bin_s_1, y_train_PO_bin_s_1)
print("Trenovaci", accuracy_score(y_train_PO_bin_s_1, xg_class_PO_s_1.predict(X_train_PO_bin_s_1)))
print("Testovaci", accuracy_score(y_test_PO_bin_s_1, xg_class_PO_s_1.predict(X_test_PO_bin_s_1)))

# Uložení vytvořeného modelu do filesystemu
# with fs.open('.....', 'wb') as f:
#    pickle.dump(xg_class_PO_s_1, f)

# Zobrazení confusion matrix
print(confusion_matrix(y_test_PO_bin_s_1, xg_class_PO_s_1.predict(X_test_PO_bin_s_1)))
print(sum(sum(confusion_matrix(y_test_PO_bin_s_1, xg_class_PO_s_1.predict(X_test_PO_bin_s_1)))))

# grid_search(xgb.XGBClassifier,sample_dummies_PO_bin_select_1, target_PO_bin_1,50,70,10,3,5)

# 8 Kategorií (0-8,8-15,15-45,45-80,80-200,200-250,250-2000,2000-)

# Model pro PO všechny proměnné s rozbinovaným targetem
xg_class_PO_2 = xgb.XGBClassifier(n_estimators=90, max_depth=4)
xg_class_PO_2.fit(X_train_PO_bin_2, y_train_PO_bin_2)
print("Trenovaci", accuracy_score(y_train_PO_bin_2, xg_class_PO_2.predict(X_train_PO_bin_2)))
print("Testovaci", accuracy_score(y_test_PO_bin_2, xg_class_PO_2.predict(X_test_PO_bin_2)))

firmy_do_roka(X_test_PO_bin_2,y_test_PO_bin_2,'customer_key_firma',xg_class_PO_2,'anonymized_64')

# Zobrazení významnosti proměnných podle GAIN rozbinovaný target
features_PO_bin_2 = zobrazeni_promennych(X_train_PO_bin_2,xg_class_PO_2,0.006)
features_PO_bin_2 = [e for e in features_PO_bin_2 if e not in drop_chybejici_pred]

# Vytvoření trénovacího a testovacího vzorku jen z vybraných proměnných     
sample_dummies_PO_bin_select_2 = sample_dummies_PO_bin[sample_dummies_PO_bin.columns.intersection(features_PO_bin_2)]
X_train_PO_bin_s_2, X_test_PO_bin_s_2, y_train_PO_bin_s_2, y_test_PO_bin_s_2 = train_test_split(sample_dummies_PO_bin_select_2, target_PO_bin_2, random_state = 0)

# Model pro PO vybrané proměnné s rozbinovaným targetem, R^2 crossvalidační metodou 0,94
xg_class_PO_s_2 = xgb.XGBClassifier(n_estimators=90, max_depth=4)
xg_class_PO_s_2.fit(X_train_PO_bin_s_2, y_train_PO_bin_s_2)
print("Trenovaci", accuracy_score(y_train_PO_bin_s_2, xg_class_PO_s_2.predict(X_train_PO_bin_s_2)))
print("Testovaci", accuracy_score(y_test_PO_bin_s_2, xg_class_PO_s_2.predict(X_test_PO_bin_s_2)))

# with fs.open('....', 'wb') as f:
#    pickle.dump(xg_class_PO_s_2, f)

print(confusion_matrix(y_test_PO_bin_s_2, xg_class_PO_s_2.predict(X_test_PO_bin_s_2)))
print(sum(sum(confusion_matrix(y_test_PO_bin_s_2, xg_class_PO_s_2.predict(X_test_PO_bin_s_2)))))

# grid_search(xgb.XGBClassifier,sample_dummies_PO_bin_select_2, target_PO_bin_2,50,600,40,1,6)

# Logaritmovaný target

# Model pro PO všechny proměnné logaritmovaný target
drop = ['FOP','target','target_log','anonymized_63','anonymized_64']

sample_dummies_PO = sample_dummies.loc[PO]
target_PO = sample_dummies.loc[PO]['target_log']
sample_dummies_PO = sample_dummies.loc[PO].drop(columns=drop)

X_train_PO, X_test_PO, y_train_PO, y_test_PO = train_test_split(sample_dummies_PO, target_PO, random_state = 0)

# Model pro PO všechny proměnné
xg_reg_PO = xgb.XGBRegrFOP_48r(n_estimators=500, max_depth=5)
xg_reg_PO.fit(X_train_PO, y_train_PO)
print("Trenovaci", r2_score(y_train_PO, xg_reg_PO.predict(X_train_PO)))
print("Testovaci", r2_score(y_test_PO, xg_reg_PO.predict(X_test_PO)))

# Test, zda model funguje i na firmách, které jsou klienti méně než rok
# firmy_do_roka(X_test_PO,y_test_PO,'customer_key_firma',xg_reg_PO,'target_log')

# Zobrazení významnosti proměnných podle GAIN rozbinovaný target
features_PO = zobrazeni_promennych(X_train_PO,xg_reg_PO,0.0065)
features_PO = [e for e in features_PO if e not in drop_chybejici_pred]
    
# Vytvoření trénovacího a testovacího vzorku jen z vybraných proměnných     
sample_dummies_PO_select = sample_dummies_PO[sample_dummies_PO.columns.intersection(features_PO)]
X_train_PO_s, X_test_PO_s, y_train_PO_s, y_test_PO_s = train_test_split(sample_dummies_PO_select, target_PO, random_state = 0)

# Model pro PO vybrané proměnné podle GAIN, R^2 na crossvalidation 0,90
xg_reg_PO_s = xgb.XGBRegrFOP_48r(n_estimators=500, max_depth=5) # parametry odhadnuté pomocí cross validation metody 
xg_reg_PO_s.fit(X_train_PO_s, y_train_PO_s)
print("Trenovaci výběr proměnných", r2_score(y_train_PO_s, xg_reg_PO_s.predict(X_train_PO_s)))
print("Testovaci výběr proměnných", r2_score(y_test_PO_s, xg_reg_PO_s.predict(X_test_PO_s)))

# with fs.open('....', 'wb') as f:
#    pickle.dump(xg_reg_PO_s, f)

# odlogaritmování targetu a predikce
y_PO = np.exp(y_test_PO_s)
y_predict_PO = np.exp(xg_reg_PO_s.predict(X_test_PO_s))
diff = (y_predict_PO-y_PO)/y_PO+1

dohromadyPO = pd.DataFrame(zip(y_PO, y_predict_PO, diff))
dohromadyPO = dohromadyPO.sort_values(by=[2])
# pd.merge(dohromady,test['stari_firmy'], left_index=True, right_index=True)
dohromadyPO

plt.figure(figsize=(10, 10))
sns.scatterplot(y_PO, y_predict_PO, s=5, linewidth=0)

# plt.scatter(y, y_predict)
plt.ylim(0, 1e9)
plt.xlim(0, 1e9)
# plt.gcf().set_size_inches((10, 10)) 

# grid_search(xgb.XGBRegrFOP_48r,sample_dummies_PO_select, target_PO,500,800,10,5,8)

# ------------------------------------------FOP------------------------------------------

# ROZBINOVANÝ TARGET

drop_FOP = ['FOP','target','target_log','anonymized_63','anonymized_64','turnover_18','turnover_17']

sample_dummies_FOP = sample_dummies.loc[FOP][ROK_2018]
target_FOP_bin_1 = sample_dummies.loc[FOP][ROK_2018]['anonymized_63']
target_FOP_bin_2 = sample_dummies.loc[FOP][ROK_2018]['anonymized_64']
sample_dummies_FOP_bin = sample_dummies_FOP.drop(columns=drop_FOP)

X_train_FOP_bin_1, X_test_FOP_bin_1, y_train_FOP_bin_1, y_test_FOP_bin_1 = train_test_split(sample_dummies_FOP_bin, target_FOP_bin_1, random_state = 0)
X_train_FOP_bin_2, X_test_FOP_bin_2, y_train_FOP_bin_2, y_test_FOP_bin_2 = train_test_split(sample_dummies_FOP_bin, target_FOP_bin_2, random_state = 0)

# 5 Kategorií (0-10,10-100,100-220,220-2000,2000-)

# Model pro FOP všechny proměnné s rozbinovaným targetem
xg_class_FOP_1 = xgb.XGBClassifier(n_estimators=130, max_depth=3)
xg_class_FOP_1.fit(X_train_FOP_bin_1, y_train_FOP_bin_1)
print("Trenovaci", accuracy_score(y_train_FOP_bin_1, xg_class_FOP_1.predict(X_train_FOP_bin_1)))
print("Testovaci", accuracy_score(y_test_FOP_bin_1, xg_class_FOP_1.predict(X_test_FOP_bin_1)))

firmy_do_roka(X_test_FOP_bin_1,y_test_FOP_bin_1,'customer_key_firma',xg_class_FOP_1,'anonymized_63')

# Zobrazení významnosti proměnných podle GAIN rozbinovaný target
features_FOP_bin_1 = zobrazeni_promennych(X_train_FOP_bin_1,xg_class_FOP_1,0.01)
features_FOP_bin_1 = [e for e in features_FOP_bin_1 if e not in drop_chybejici_pred]

# Vytvoření trénovacího a testovacího vzorku jen z vybraných proměnných     
sample_dummies_FOP_bin_select_1 = sample_dummies_FOP_bin[sample_dummies_FOP_bin.columns.intersection(features_FOP_bin_1)]
X_train_FOP_bin_s_1, X_test_FOP_bin_s_1, y_train_FOP_bin_s_1, y_test_FOP_bin_s_1 = train_test_split(sample_dummies_FOP_bin_select_1, target_FOP_bin_1, random_state = 0)

# Model pro FOP vybrané proměnné s rozbinovaným targetem, R^2 crossvalidační metodou 0,94
xg_class_FOP_s_1 = xgb.XGBClassifier(n_estimators=130, max_depth=3)
xg_class_FOP_s_1.fit(X_train_FOP_bin_s_1, y_train_FOP_bin_s_1)
print("Trenovaci", accuracy_score(y_train_FOP_bin_s_1, xg_class_FOP_s_1.predict(X_train_FOP_bin_s_1)))
print("Testovaci", accuracy_score(y_test_FOP_bin_s_1, xg_class_FOP_s_1.predict(X_test_FOP_bin_s_1)))

# with fs.open('....', 'wb') as f:
#    pickle.dump(xg_class_FOP_s_1, f)

print(confusion_matrix(y_test_FOP_bin_s_1, xg_class_FOP_s_1.predict(X_test_FOP_bin_s_1)))
print(sum(sum(confusion_matrix(y_test_FOP_bin_s_1, xg_class_FOP_s_1.predict(X_test_FOP_bin_s_1)))))

# grid_search(xgb.XGBClassifier,sample_dummies_FOP_bin_select_1, target_FOP_bin_1,50,600,40,1,6)

# 8 Kategorií (0-8,8-15,15-45,45-80,80-200,200-250,250-2000,2000-)

# Model pro FOP všechny proměnné s rozbinovaným targetem
xg_class_FOP_2 = xgb.XGBClassifier(n_estimators=130, max_depth=3)
xg_class_FOP_2.fit(X_train_FOP_bin_2, y_train_FOP_bin_2)
print("Trenovaci", accuracy_score(y_train_FOP_bin_2, xg_class_FOP_2.predict(X_train_FOP_bin_2)))
print("Testovaci", accuracy_score(y_test_FOP_bin_2, xg_class_FOP_2.predict(X_test_FOP_bin_2)))

firmy_do_roka(X_test_FOP_bin_2,y_test_FOP_bin_2,'customer_key_firma',xg_class_FOP_2,'anonymized_64')

# Zobrazení významnosti proměnných podle GAIN rozbinovaný target
features_FOP_bin_2 = zobrazeni_promennych(X_train_FOP_bin_2,xg_class_FOP_2,0.01)
features_FOP_bin_2 = [e for e in features_FOP_bin_2 if e not in drop_chybejici_pred]

# Vytvoření trénovacího a testovacího vzorku jen z vybraných proměnných     
sample_dummies_FOP_bin_select_2 = sample_dummies_FOP_bin[sample_dummies_FOP_bin.columns.intersection(features_FOP_bin_2)]
X_train_FOP_bin_s_2, X_test_FOP_bin_s_2, y_train_FOP_bin_s_2, y_test_FOP_bin_s_2 = train_test_split(sample_dummies_FOP_bin_select_2, target_FOP_bin_2, random_state = 0)

# Model pro FOP vybrané proměnné s rozbinovaným targetem, R^2 crossvalidační metodou 0,94
xg_class_FOP_s_2 = xgb.XGBClassifier(n_estimators=130, max_depth=3)
xg_class_FOP_s_2.fit(X_train_FOP_bin_s_2, y_train_FOP_bin_s_2)
print("Trenovaci", accuracy_score(y_train_FOP_bin_s_2, xg_class_FOP_s_2.predict(X_train_FOP_bin_s_2)))
print("Testovaci", accuracy_score(y_test_FOP_bin_s_2, xg_class_FOP_s_2.predict(X_test_FOP_bin_s_2)))

# with fs.open('....', 'wb') as f:
#    pickle.dump(xg_class_FOP_s_2, f)

print(confusion_matrix(y_test_FOP_bin_s_2, xg_class_FOP_s_2.predict(X_test_FOP_bin_s_2)))
print(sum(sum(confusion_matrix(y_test_FOP_bin_s_2, xg_class_FOP_s_2.predict(X_test_FOP_bin_s_2)))))

# grid_search(xgb.XGBClassifier,sample_dummies_FOP_bin_select_2, target_FOP_bin_2,50,600,40,1,6)

# LOGARITMOVANÝ TARGET

drop_FOP = ['FOP','target','target_log','anonymized_63','anonymized_64','anonymized_61','turnover_18','turnover_17','ptp_statsubs_cat_cat_Klient']

sample_dummies_FOP = sample_dummies.loc[FOP]
target_FOP = sample_dummies.loc[FOP]['target_log']
sample_dummies_FOP = sample_dummies_FOP.drop(columns=drop_FOP)

X_train_FOP, X_test_FOP, y_train_FOP, y_test_FOP = train_test_split(sample_dummies_FOP, target_FOP)

xg_reg_FOP = xgb.XGBRegrFOP_48r(n_estimators=140, max_depth=3)
xg_reg_FOP.fit(X_train_FOP, y_train_FOP)
print("Trenovaci", r2_score(y_train_FOP, xg_reg_FOP.predict(X_train_FOP)))
print("Testovaci", r2_score(y_test_FOP, xg_reg_FOP.predict(X_test_FOP)))

# Test, zda model funguje i na firmách, které jsou klienti méně než rok
# firmy_do_roka(X_test_FOP,y_test_FOP,'customer_key_firma',xg_reg_FOP,'target_log')

# Zobrazení významnosti proměnných podle GAIN rozbinovaný target
features_FOP = zobrazeni_promennych(X_train_FOP,xg_reg_FOP,0.006)
features_FOP = [e for e in features_FOP if e not in drop_chybejici_pred]
     
# Vytvoření trénovacího a testovacího vzorku jen z vybraných proměnných    
sample_dummies_FOP_select = sample_dummies_FOP[sample_dummies_FOP.columns.intersection(features_FOP)]
X_train_FOP_s, X_test_FOP_s, y_train_FOP_s, y_test_FOP_s = train_test_split(sample_dummies_FOP_select, target_FOP)

# Model pro PO vybrané proměnné podle GAIN, R^2 na crossvalidation 0,83
xg_reg_FOP_s = xgb.XGBRegrFOP_48r(n_estimators=140, max_depth=3) # parametry odhadnuté pomocí cross validation metody 
xg_reg_FOP_s.fit(X_train_FOP_s, y_train_FOP_s)
print("Trenovaci výběr proměnných", r2_score(y_train_FOP_s, xg_reg_FOP_s.predict(X_train_FOP_s)))
print("Testovaci výběr proměnných", r2_score(y_test_FOP_s, xg_reg_FOP_s.predict(X_test_FOP_s)))

# with fs.open('....', 'wb') as f:
#    pickle.dump(xg_reg_FOP_s, f)

# odlogaritmování targetu a predikce
y_FOP = np.exp(y_test_FOP_s)
y_predict_FOP = np.exp(xg_reg_FOP_s.predict(X_test_FOP_s))
diff = (y_predict_FOP-y_FOP)/y_FOP+1

dohromadyFOP = pd.DataFrame(zip(y_FOP, y_predict_FOP, diff))
dohromadyFOP = dohromadyFOP.sort_values(by=[2])
# pd.merge(dohromady,test['stari_firmy'], left_index=True, right_index=True)
dohromadyFOP

plt.figure(figsize=(10, 10))
sns.scatterplot(y_FOP, y_predict_FOP, s=5, linewidth=0)

# plt.scatter(y, y_predict)
plt.ylim(0, 1e7)
plt.xlim(0, 1e7)
# plt.gcf().set_size_inches((10, 10)) 

# grid_search(xgb.XGBRegrFOP_48r,sample_dummies_FOP_select, target_FOP,50,600,40,1,6)

# Modely - pouze neklientská data

# ------------------------------------------PO------------------------------------------

# Výběr pouze proměnných, které jsou k dispozici pro klienty i neklienty. Všechny proměnné, které jsou napočítané nad klientskými daty se zde vyloučí
sample_dummies_PO_nekl = sample_dummies.loc[PO][['anonymized_60','anonymized_61','anonymized_63', 'anonymized_64','anonymized_65','anonymized_66','anonymized_67','anonymized_68','anonymized_69','anonymized_70','anonymized_71','anonymized_72','anonymized_73','anonymized_74','anonymized_75','anonymized_76','anonymized_77','anonymized_78','anonymized_79','anonymized_80']]

# Model pro PO všechny proměnné rozbinovaný target
drop = ['anonymized_63','anonymized_64']

sample_dummies_PO_nekl = sample_dummies_PO_nekl.loc[PO]
target_PO_bin_1_nekl = sample_dummies_PO_nekl['anonymized_63']
target_PO_bin_2_nekl = sample_dummies_PO_nekl['anonymized_64']
sample_dummies_PO_bin_nekl = sample_dummies_PO_nekl.drop(columns=drop)

X_train_PO_bin_1_nekl, X_test_PO_bin_1_nekl, y_train_PO_bin_1_nekl, y_test_PO_bin_1_nekl = train_test_split(sample_dummies_PO_bin_nekl, target_PO_bin_1_nekl,random_state=0)
X_train_PO_bin_2_nekl, X_test_PO_bin_2_nekl, y_train_PO_bin_2_nekl, y_test_PO_bin_2_nekl = train_test_split(sample_dummies_PO_bin_nekl, target_PO_bin_2_nekl,random_state=0)

# 5 Kategorií (0-10,10-100,100-220,220-2000,2000-)

xg_class_PO_1_nekl = xgb.XGBClassifier(n_estimators=50, max_depth=2)
xg_class_PO_1_nekl.fit(X_train_PO_bin_1_nekl, y_train_PO_bin_1_nekl)
print("Trenovaci", accuracy_score(y_train_PO_bin_1_nekl, xg_class_PO_1_nekl.predict(X_train_PO_bin_1_nekl)))
print("Testovaci", accuracy_score(y_test_PO_bin_1_nekl, xg_class_PO_1_nekl.predict(X_test_PO_bin_1_nekl)))

# Zobrazení významnosti proměnných podle GAIN rozbinovaný target
features_PO_bin_1_nekl = zobrazeni_promennych(X_train_PO_bin_1_nekl,xg_class_PO_1_nekl,0.002)
# Vytvoření trénovacího a testovacího vzorku jen z vybraných proměnných  
sample_dummies_PO_bin_select_1_nekl = sample_dummies_PO_nekl[sample_dummies_PO_bin_nekl.columns.intersection(features_PO_bin_1_nekl.tolist())]
X_train_PO_bin_s_1_nekl, X_test_PO_bin_s_1_nekl, y_train_PO_bin_s_1_nekl, y_test_PO_bin_s_1_nekl = train_test_split(sample_dummies_PO_bin_select_1_nekl, target_PO_bin_1_nekl, random_state = 0)

# Model pro PO vybrané proměnné s rozbinovaným targetem, R^2 crossvalidační metodou 0,94
xg_class_PO_s_1_nekl = xgb.XGBClassifier(n_estimators=50, max_depth=2)
xg_class_PO_s_1_nekl.fit(X_train_PO_bin_s_1_nekl, y_train_PO_bin_s_1_nekl)
print("Trenovaci", accuracy_score(y_train_PO_bin_s_1_nekl, xg_class_PO_s_1_nekl.predict(X_train_PO_bin_s_1_nekl)))
print("Testovaci", accuracy_score(y_test_PO_bin_s_1_nekl, xg_class_PO_s_1_nekl.predict(X_test_PO_bin_s_1_nekl)))

# with fs.open('....', 'wb') as f:
#    pickle.dump(xg_class_PO_s_1_nekl, f)

print(confusion_matrix(y_test_PO_bin_s_1_nekl, xg_class_PO_s_1_nekl.predict(X_test_PO_bin_s_1_nekl)))
print(sum(sum(confusion_matrix(y_test_PO_bin_s_1_nekl, xg_class_PO_s_1_nekl.predict(X_test_PO_bin_s_1_nekl)))))

# grid_search(xgb.XGBClassifier,sample_dummies_PO_bin_select_1_nekl, target_PO_bin_1_nekl,50,600,40,1,6)

# 8 Kategorií (0-8,8-15,15-45,45-80,80-200,200-250,250-2000,2000-)

xg_class_PO_2_nekl = xgb.XGBClassifier(n_estimators=130, max_depth=2)
xg_class_PO_2_nekl.fit(X_train_PO_bin_2_nekl, y_train_PO_bin_2_nekl)
print("Trenovaci", accuracy_score(y_train_PO_bin_2_nekl, xg_class_PO_2_nekl.predict(X_train_PO_bin_2_nekl)))
print("Testovaci", accuracy_score(y_test_PO_bin_2_nekl, xg_class_PO_2_nekl.predict(X_test_PO_bin_2_nekl))) 

# Zobrazení významnosti proměnných podle GAIN rozbinovaný target
features_PO_bin_2_nekl = zobrazeni_promennych(X_train_PO_bin_2_nekl,xg_class_PO_2_nekl,0.005)
# Vytvoření trénovacího a testovacího vzorku jen z vybraných proměnných  
sample_dummies_PO_bin_select_2_nekl = sample_dummies_PO_nekl[sample_dummies_PO_bin_nekl.columns.intersection(features_PO_bin_2_nekl.tolist())]
X_train_PO_bin_s_2_nekl, X_test_PO_bin_s_2_nekl, y_train_PO_bin_s_2_nekl, y_test_PO_bin_s_2_nekl = train_test_split(sample_dummies_PO_bin_select_2_nekl, target_PO_bin_2_nekl, random_state = 0)

# Model pro PO vybrané proměnné s rozbinovaným targetem, R^2 crossvalidační metodou 0,94
xg_class_PO_s_2_nekl = xgb.XGBClassifier(n_estimators=130, max_depth=2)
xg_class_PO_s_2_nekl.fit(X_train_PO_bin_s_2_nekl, y_train_PO_bin_s_2_nekl)
print("Trenovaci", accuracy_score(y_train_PO_bin_s_2_nekl, xg_class_PO_s_2_nekl.predict(X_train_PO_bin_s_2_nekl)))
print("Testovaci", accuracy_score(y_test_PO_bin_s_2_nekl, xg_class_PO_s_2_nekl.predict(X_test_PO_bin_s_2_nekl)))

# with fs.open('....', 'wb') as f:
#    pickle.dump(xg_class_PO_s_2_nekl, f)

print(confusion_matrix(y_test_PO_bin_s_2_nekl, xg_class_PO_s_2_nekl.predict(X_test_PO_bin_s_2_nekl)))
print(sum(sum(confusion_matrix(y_test_PO_bin_s_2_nekl, xg_class_PO_s_2_nekl.predict(X_test_PO_bin_s_2_nekl)))))                                    

# grid_search(xgb.XGBClassifier,sample_dummies_PO_bin_select_2_nekl, target_PO_bin_2_nekl,50,600,40,1,6)

# ------------------------------------------FOP------------------------------------------

sample_dummies_FOP_nekl = sample_dummies.loc[FOP][['anonymized_60','anonymized_61','anonymized_63', 'anonymized_64','anonymized_65','anonymized_66','anonymized_67','anonymized_68','anonymized_69','anonymized_70','anonymized_71','anonymized_72','anonymized_73','anonymized_74','anonymized_75','anonymized_76','anonymized_77','anonymized_78','anonymized_79','anonymized_80']]

# Model pro FOP všechny proměnné rozbinovaný target
drop = ['anonymized_63','anonymized_64']

sample_dummies_FOP_nekl = sample_dummies_FOP_nekl.loc[FOP][ROK_2018]
target_FOP_bin_1_nekl = sample_dummies_FOP_nekl[ROK_2018]['anonymized_63']
target_FOP_bin_2_nekl = sample_dummies_FOP_nekl[ROK_2018]['anonymized_64']
sample_dummies_FOP_bin_nekl = sample_dummies_FOP_nekl.drop(columns=drop)

X_train_FOP_bin_1_nekl, X_test_FOP_bin_1_nekl, y_train_FOP_bin_1_nekl, y_test_FOP_bin_1_nekl = train_test_split(sample_dummies_FOP_bin_nekl, target_FOP_bin_1_nekl, random_state = 0)
X_train_FOP_bin_2_nekl, X_test_FOP_bin_2_nekl, y_train_FOP_bin_2_nekl, y_test_FOP_bin_2_nekl = train_test_split(sample_dummies_FOP_bin_nekl, target_FOP_bin_2_nekl, random_state = 0)

# 5 Kategorií (0-10,10-100,100-220,220-2000,2000-)

xg_class_FOP_1_nekl = xgb.XGBClassifier(n_estimators=90, max_depth=1)
xg_class_FOP_1_nekl.fit(X_train_FOP_bin_1_nekl, y_train_FOP_bin_1_nekl)
print("Trenovaci", accuracy_score(y_train_FOP_bin_1_nekl, xg_class_FOP_1_nekl.predict(X_train_FOP_bin_1_nekl)))
print("Testovaci", accuracy_score(y_test_FOP_bin_1_nekl, xg_class_FOP_1_nekl.predict(X_test_FOP_bin_1_nekl)))

# Zobrazení významnosti proměnných FOPdle GAIN rozbinovaný target
features_FOP_bin_1_nekl = zobrazeni_promennych(X_train_FOP_bin_1_nekl,xg_class_FOP_1_nekl,0.005)
# Vytvoření trénovacího a testovacího vzorku jen z vybraných proměnných  
sample_dummies_FOP_bin_select_1_nekl = sample_dummies_FOP_nekl[sample_dummies_FOP_bin_nekl.columns.intersection(features_FOP_bin_1_nekl.tolist())]
X_train_FOP_bin_s_1_nekl, X_test_FOP_bin_s_1_nekl, y_train_FOP_bin_s_1_nekl, y_test_FOP_bin_s_1_nekl = train_test_split(sample_dummies_FOP_bin_select_1_nekl, target_FOP_bin_1_nekl, random_state = 0)

# Model pro FOP vybrané proměnné s rozbinovaným targetem, R^2 crossvalidační metodou 0,94
xg_class_FOP_s_1_nekl = xgb.XGBClassifier(n_estimators=90, max_depth=1)
xg_class_FOP_s_1_nekl.fit(X_train_FOP_bin_s_1_nekl, y_train_FOP_bin_s_1_nekl)
print("Trenovaci", accuracy_score(y_train_FOP_bin_s_1_nekl, xg_class_FOP_s_1_nekl.predict(X_train_FOP_bin_s_1_nekl)))
print("Testovaci", accuracy_score(y_test_FOP_bin_s_1_nekl, xg_class_FOP_s_1_nekl.predict(X_test_FOP_bin_s_1_nekl)))

# with fs.open('....', 'wb') as f:
#    pickle.dump(xg_class_FOP_s_1_nekl, f)

print(confusion_matrix(y_test_FOP_bin_s_1_nekl, xg_class_FOP_s_1_nekl.predict(X_test_FOP_bin_s_1_nekl)))
print(sum(sum(confusion_matrix(y_test_FOP_bin_s_1_nekl, xg_class_FOP_s_1_nekl.predict(X_test_FOP_bin_s_1_nekl)))))

# grid_search(xgb.XGBClassifier,sample_dummies_FOP_bin_select_1_nekl, target_FOP_bin_1_nekl,50,600,40,1,6)

# 8 Kategorií (0-8,8-15,15-45,45-80,80-200,200-250,250-2000,2000-)

xg_class_FOP_2_nekl = xgb.XGBClassifier(n_estimators=30, max_depth=2)
xg_class_FOP_2_nekl.fit(X_train_FOP_bin_2_nekl, y_train_FOP_bin_2_nekl)
print("Trenovaci", accuracy_score(y_train_FOP_bin_2_nekl, xg_class_FOP_2_nekl.predict(X_train_FOP_bin_2_nekl)))
print("Testovaci", accuracy_score(y_test_FOP_bin_2_nekl, xg_class_FOP_2_nekl.predict(X_test_FOP_bin_2_nekl)))

# Zobrazení významnosti proměnných FOPdle GAIN rozbinovaný target
features_FOP_bin_2_nekl = zobrazeni_promennych(X_train_FOP_bin_2_nekl,xg_class_FOP_2_nekl,0.005)
# Vytvoření trénovacího a testovacího vzorku jen z vybraných proměnných  
sample_dummies_FOP_bin_select_2_nekl = sample_dummies_FOP_nekl[sample_dummies_FOP_bin_nekl.columns.intersection(features_FOP_bin_2_nekl.tolist())]
X_train_FOP_bin_s_2_nekl, X_test_FOP_bin_s_2_nekl, y_train_FOP_bin_s_2_nekl, y_test_FOP_bin_s_2_nekl = train_test_split(sample_dummies_FOP_bin_select_2_nekl, target_FOP_bin_2_nekl, random_state = 0)

# Model pro FOP vybrané proměnné s rozbinovaným targetem, R^2 crossvalidační metodou 0,94
xg_class_FOP_s_2_nekl = xgb.XGBClassifier(n_estimators=30, max_depth=2)
xg_class_FOP_s_2_nekl.fit(X_train_FOP_bin_s_2_nekl, y_train_FOP_bin_s_2_nekl)
print("Trenovaci", accuracy_score(y_train_FOP_bin_s_2_nekl, xg_class_FOP_s_2_nekl.predict(X_train_FOP_bin_s_2_nekl)))
print("Testovaci", accuracy_score(y_test_FOP_bin_s_2_nekl, xg_class_FOP_s_2_nekl.predict(X_test_FOP_bin_s_2_nekl)))

# with fs.open('....', 'wb') as f:
#    pickle.dump(xg_class_FOP_s_2_nekl, f)

print(confusion_matrix(y_test_FOP_bin_s_2_nekl, xg_class_FOP_s_2_nekl.predict(X_test_FOP_bin_s_2_nekl)))
print(sum(sum(confusion_matrix(y_test_FOP_bin_s_2_nekl, xg_class_FOP_s_2_nekl.predict(X_test_FOP_bin_s_2_nekl)))))

# grid_search(xgb.XGBClassifier,sample_dummies_FOP_bin_select_2_nekl, target_FOP_bin_2_nekl,50,600,40,1,6)

# -----------------------------------SKOROVÁNÍ---------------------------------

# Je potřeba importovaný seznam prediktorů omezit, jinak by import trval velmi dlouho a nemusela by stačit paměť
features_all = pd.DataFrame(np.unique(np.array(list(chain.from_iterable([features_PO_bin_1,features_PO_bin_2,features_PO,features_FOP_bin_1,features_FOP_bin_2,features_FOP,features_PO_bin_1_nekl,features_PO_bin_2_nekl,features_FOP_bin_1_nekl,features_FOP_bin_2_nekl])))))
rusb_load(features_all, 'features_obrat')

# načtení uložených modelů
#     f1.download('model_tmp1')
#     f2.download('model_tmp2')
#     f3.download('model_tmp3')
#     f4.download('model_tmp4')
#     f5.download('model_tmp5')
#     f6.download('model_tmp6')
#     f7.download('model_tmp7')
#     f8.download('model_tmp8')
#     f9.download('model_tmp9')
#     f10.download('model_tmp10')
    
#     xg_class_FOP_s_1 = pickle.load(f1)
#     xg_class_FOP_s_1_nekl = pickle.load(f2)
#     xg_class_FOP_s_2 = pickle.load(f3)
#     xg_class_FOP_s_2_nekl = pickle.load(f4)
#     xg_class_PO_s_1 = pickle.load(f5)
#     xg_class_PO_s_1_nekl = pickle.load(f6)
#     xg_class_PO_s_2 = pickle.load(f7)
#     xg_class_PO_s_2_nekl = pickle.load(f8)
#     xg_reg_FOP_s = pickle.load(f9)
#     xg_reg_PO_s = pickle.load(f10)

#  populace pro skórování
population = %sql select * from source.data_20191231
population = population.set_index('customer_key')

population['FOP'].fillna('N',inplace=True)

FOP_KL =     (population['FOP'] == 'Y') & (population['klient'] == 'Y')
FOP_NEKL =   (population['FOP'] == 'Y') & (population['klient'] == 'N')
PO_KL =      (population['FOP'] == 'N') & (population['klient'] == 'Y')
PO_NEKL =    (population['FOP'] == 'N') & (population['klient'] == 'N')

# rozdělení na 4 skupiny
population_kl_PO = population.loc[PO_KL]
population_kl_FOP = population.loc[FOP_KL]

population_nekl_PO = population.loc[PO_NEKL]
population_nekl_FOP = population.loc[FOP_NEKL]

population_kl_PO_1 = population_kl_PO.apply(pd.to_numeric, errors='coerce', axis=1)
population_kl_FOP_1 = population_kl_FOP.apply(pd.to_numeric, errors='coerce', axis=1)

population_nekl_PO_1 = population_nekl_PO.apply(pd.to_numeric, errors='coerce', axis=1)
population_nekl_FOP_1 = population_nekl_FOP.apply(pd.to_numeric, errors='coerce', axis=1)

# selekce správných prediktorů pro jednotlivé modely
dataset_PO_bin_1 = population_kl_PO_1[xg_class_PO_s_1.get_booster().feature_names]
dataset_PO_bin_2 = population_kl_PO_1[xg_class_PO_s_2.get_booster().feature_names]
dataset_PO = population_kl_PO_1[xg_reg_PO_s.get_booster().feature_names]

dataset_FOP_bin_1 = population_kl_FOP_1[xg_class_FOP_s_1.get_booster().feature_names]
dataset_FOP_bin_2 = population_kl_FOP_1[xg_class_FOP_s_2.get_booster().feature_names]
dataset_FOP = population_kl_FOP_1[xg_reg_FOP_s.get_booster().feature_names]

dataset_PO_bin_1_nekl = population_nekl_PO_1[xg_class_PO_s_1_nekl.get_booster().feature_names]
dataset_PO_bin_2_nekl = population_nekl_PO_1[xg_class_PO_s_2_nekl.get_booster().feature_names]

dataset_FOP_bin_1_nekl = population_nekl_FOP_1[xg_class_FOP_s_1_nekl.get_booster().feature_names]
dataset_FOP_bin_2_nekl = population_nekl_FOP_1[xg_class_FOP_s_2_nekl.get_booster().feature_names]

PO_kl_predikce = (pd.concat
            (
                [
                    pd.Series(population_kl_PO.index), 
                    pd.Series(xg_class_PO_s_1.predict(dataset_PO_bin_1)),
                    pd.Series(xg_class_PO_s_2.predict(dataset_PO_bin_2)),
                    pd.Series(np.exp(xg_reg_PO_s.predict(dataset_PO)))
                ], axis=1
            )
        )

FOP_kl_predikce = (pd.concat
            (
                [
                    pd.Series(population_kl_FOP.index), 
                    pd.Series(xg_class_FOP_s_1.predict(dataset_FOP_bin_1)),
                    pd.Series(xg_class_FOP_s_2.predict(dataset_FOP_bin_2)),
                    pd.Series(np.exp(xg_reg_FOP_s.predict(dataset_FOP)))
                ], axis=1
            )
        )

PO_nekl_predikce = (pd.concat
            (
                [
                    pd.Series(population_nekl_PO.index), 
                    pd.Series(xg_class_PO_s_1_nekl.predict(dataset_PO_bin_1_nekl)),
                    pd.Series(xg_class_PO_s_2_nekl.predict(dataset_PO_bin_2_nekl))
                ], axis=1
            )
        )

FOP_nekl_predikce = (pd.concat
            (
                [
                    pd.Series(population_nekl_FOP.index), 
                    pd.Series(xg_class_FOP_s_1_nekl.predict(dataset_FOP_bin_1_nekl)),
                    pd.Series(xg_class_FOP_s_2_nekl.predict(dataset_FOP_bin_2_nekl))
                ], axis=1
            )
        )

PO_kl_predikce = PO_kl_predikce.rename(columns={0: "kategorii_5", 1: "kategorii_8", 2: "presna_hodnota"})
FOP_kl_predikce = FOP_kl_predikce.rename(columns={0: "kategorii_5", 1: "kategorii_8", 2: "presna_hodnota"})
PO_nekl_predikce = PO_nekl_predikce.rename(columns={0: "kategorii_5", 1: "kategorii_8"})
FOP_nekl_predikce = FOP_nekl_predikce.rename(columns={0: "kategorii_5", 1: "kategorii_8"})


# /********************************** Úprava výsledků modelu **********************************/

PO_nekl_predikce = PO_nekl_predikce.set_index('customer_key')
PO_nekl_predikce_oprava = dataset_PO_bin_1_nekl.join(PO_nekl_predikce)
PO_nekl_predikce_oprava.loc[pd.isnull(PO_nekl_predikce_oprava['anonymized_61']), 'kategorii_5_opr'] = 1
PO_nekl_predikce_oprava["kategorii_5_opr_opr"] = PO_nekl_predikce_oprava["kategorii_5_opr"].fillna(PO_nekl_predikce_oprava["kategorii_5"])
PO_nekl_predikce['kategorii_5'] = PO_nekl_predikce_oprava["kategorii_5_opr_opr"]
PO_nekl_predikce['customer_key'] = PO_nekl_predikce.index



predikce_obrat_vse =  pd.concat([PO_kl_predikce,FOP_kl_predikce,PO_nekl_predikce,FOP_nekl_predikce])

predikce_obrat_vse

predikce_obrat_vse.groupby(by=["kategorii_5"]).count()
























